In [1]:
import string
import Levenshtein as lev
from ete3 import Tree
from pysam import FastaFile

In [2]:
def distance_lev(str1,str2):
    distance_strings = lev.distance(str1,str2)
    return distance_strings

distance_lev('king','kind')
distance_lev('ki-g','jkkl-d')

4

In [16]:
# load the tree
tree_string_file = open('/Users/sanjanatule/Documents/uq/Projects/PreferredPath/data/sample_tree/grasp_ancestors.nwk',"r")
my_tree = tree_string_file.read() + ";"
t = Tree(my_tree, format=1)

In [4]:
#print(t)

In [21]:
# load the ancestor sequences
#ancestor_fasta = "/Users/sanjanatule/Documents/uq/Projects/PreferredPath/data/sample_tree/grasp_ancestors.fa"
#ancestors_sequences_info = FastaFile(ancestor_fasta)

ancestor_fasta = "/Users/sanjanatule/Documents/uq/Projects/PreferredPath/data/sample_tree/new_grasp_ancestors.fasta"
ancestors_sequences_info = FastaFile(ancestor_fasta)

# load extant sequences
input_file = "/Users/sanjanatule/Documents/uq/Projects/PreferredPath/data/sample_tree/input_extants.fasta"
extant_sequences_info = FastaFile(input_file)

In [22]:
# load sequences on the tree
for n in t.traverse():
    try:
        n.add_features(sequence = ancestors_sequences_info.fetch(n.name))
    except:
        n.add_features(sequence = extant_sequences_info.fetch(n.name))

In [23]:
# traverse tree from top down and append 3 sequences ( except for root) for each node to calculate the Lev distance 
# from its neighbours
for n in t.traverse():
#     print("node-",n.name)
    if n.is_root() == True: # has no parent
        neighbour_sequence_list = []
        for c in n.children:
            neighbour_sequence_list.append(c.sequence)
            
        n.add_features(neighbour_sequence = neighbour_sequence_list)
    
    elif n.is_leaf() == False:
        neighbour_sequence_list = []
        # child sequences
        for c in n.children:
            neighbour_sequence_list.append(c.sequence)
        # parent sequences
        neighbour_sequence_list.append(n.up.sequence)
        n.add_features(neighbour_sequence = neighbour_sequence_list)

In [24]:
# calculate the average distance for each node
parsi_score = 0
for n in t.traverse():
    if n.is_leaf() == False:
        neigh_dist_list = []
        # for each neighbour calculate the distance
        for s in n.neighbour_sequence:
            neigh_dist_list.append(distance_lev(n.sequence,s))
        # average distance
        node_avg_distance = sum(neigh_dist_list) / len(neigh_dist_list) 
        parsi_score = parsi_score + node_avg_distance
print("total score::",parsi_score)

total score:: 12.0
